# Spaceship Titanic
<br>
work in progress ...<br>
adding ADA worsened LB score <br>


In [ ]:
import time
from datetime import datetime

#measure notebook running time
start_time = time.time()

%matplotlib inline

import os, warnings
import numpy as np 
from numpy.random import seed
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import metrics
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix, precision_score,recall_score, f1_score, classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from category_encoders import MEstimateEncoder

sns.set(style='white', context='notebook', palette='deep', rc={'figure.figsize':(10,8)})
print("loaded ...")

In [ ]:
# Reproducibility
def set_seed(sd):
    seed(sd)
    np.random.seed(sd)
    tf.random.set_seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
RandomSeed = 13
set_seed(RandomSeed)

# EDA

In [ ]:
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_data['Transported'] = -1
train_data['Set'] = "Train"
test_data['Set'] = "Test"
DATA = train_data.append(test_data)
DATA.reset_index(inplace=True)
DATA.info()

In [ ]:
print("Missing data:\n", DATA.isna().sum()/DATA.shape[0])

In [ ]:
DATA.head()

## PassengerId
GroupSizeCat ... small effect,
EXCLUDED

In [ ]:
DATA[['Group','Position']] = DATA[['PassengerId']].apply(lambda row: row.PassengerId.split("_"), axis = 1, result_type = 'expand')
DATA['GroupSize'] = DATA.groupby(['Group']).Group.transform('count')
DATA['GroupSizeCat'] = pd.cut(DATA['GroupSize'], bins=[1,2,8,10], labels = ['small','med', 'big'], right=False)

In [ ]:
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "GroupSize", hue="Transported");

In [ ]:
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "GroupSizeCat", hue="Transported");


## Name, Family
FamilySizeCat ... small effect;
EXCLUDED


In [ ]:
DATA[['FirstName','LastName']] = DATA[['Name']][DATA.Name.notna()].apply(lambda row: row.Name.split(" "), axis = 1, result_type = 'expand')
DATA['FamilySize'] = DATA.groupby(['LastName']).LastName.transform('count')
DATA.loc[DATA.Name.isna(),['FamilySize']] = DATA['GroupSize']

In [ ]:
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "FamilySize", hue="Transported");

In [ ]:
DATA['FamilySizeCat'] = pd.cut(DATA['FamilySize'], bins=[1,7,16,20], labels = ['small','med', 'big'], right=False)
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "FamilySizeCat", hue="Transported");

### Combination of GroupSize and FamilySize
also not useful .. <br>
EXCLUDED

In [ ]:
def change_cat(g,f):
    if g == "small" and f == "med": return 'SM'
    if f == "small" and g == "med": return 'MS'
    return "other"

DATA['ClusterSizeCat'] = DATA[['GroupSizeCat','FamilySizeCat']].apply(lambda row: change_cat(*row), axis = 1)
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "ClusterSizeCat", hue="Transported");

## Solo
worsens LB score; EXLUDED<br>


In [ ]:
DATA['Solo'] = DATA.GroupSize.apply(lambda x: 1 if x == 1 else 0)
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "Solo", hue="Transported");

## Homeplanet
Europeans are more likely to be transported, Earthings less ... <br>
INCLUDED

In [ ]:
DATA.HomePlanet = DATA.HomePlanet.fillna(DATA.HomePlanet.mode()[0])
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "HomePlanet", hue="Transported");

## CryoSleep
Those in CryoSleep are more likely to be transported<br>
INCLUDED

In [ ]:
DATA.CryoSleep = DATA.CryoSleep.fillna(DATA.CryoSleep.mode()[0])
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "CryoSleep", hue="Transported");

In [ ]:
DATA.CryoSleep = DATA.CryoSleep.astype(int)

## Cabin

In [ ]:
DATA.Cabin.fillna('A/1/P', inplace=True)
DATA[['Deck','CabinNumber','Side']] = DATA[['Cabin']].apply(lambda row: row.Cabin.split("/"), axis = 1, result_type = 'expand')

Concatenate similar decks

In [ ]:
DATA.loc[DATA.Deck.isin(['F','E']),['Deck']] = 'EF'
DATA.loc[DATA.Deck.isin(['B','C']),['Deck']] = 'BC'
DATA.loc[DATA.Deck.isin(['A','G', 'D', 'T']),['Deck']] = 'Rest'

## Deck
Deck clusters affects transport, but LB score is better only if other cabin data is included! <br>
INCLUDED

In [ ]:
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "Deck", hue="Transported");

## Side
Side affects Transport!; improves LB score <br>
INCLUDED

In [ ]:
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "Side", hue="Transported");

## Destination
Destination slightly affects transport and improves LB score<br>
INCLUDED

In [ ]:
DATA.Destination = DATA.Destination.fillna(DATA.Destination.mode()[0])
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "Destination", hue="Transported");

## Age
inreases LB score <br>
INCLUDED


In [ ]:
DATA.Age = DATA.Age.fillna(DATA.Age.mean())

In [ ]:
g = sns.FacetGrid(DATA[DATA.Set == 'Train'],col="Transported")
g = g.map(sns.histplot, "Age", kde=True)
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("GroupSize distribution over Age");

## AgeCat
Looks like infants <= 5 years are more likely to be transported.<br>
decreases LB score <br>
EXCLUDED

In [ ]:
ax = sns.displot(data=DATA[DATA.Set == 'Train'], x='Age', hue='Transported');
ax.set(xlim = (0,10));

In [ ]:
ax = sns.kdeplot(data=DATA[DATA.Set == 'Train'], x='Age', hue='Transported');
ax.set(xlim = (0));

In [ ]:
DATA['AgeCat'] = pd.cut(DATA['Age'], bins=[0,6,100], labels = ['infant','rest'], right=False)
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "AgeCat", hue="Transported");

## VIP
insignificant, VIPs are more likely to be transported, but there is so few of them .... and it worsens LB score<br>
EXCLUDED

In [ ]:
DATA.VIP = DATA.VIP.fillna(DATA.VIP.mode()[0])
sns.countplot(data=DATA[DATA.Set == 'Train'], x= "VIP", hue="Transported");

In [ ]:
DATA.VIP = DATA.VIP.astype(int)

## Luxuries
LUX and Basic are very powerful features (one is enough)<br>
INCLUDED <br>
L/B ratio worsens the score ... EXCLUDED<br>
the rest are EXLUDED<br>

In [ ]:
luxuries = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for lux in luxuries:
    DATA[lux] = DATA[lux].fillna(DATA[lux].mean())

In [ ]:
extras = ['RoomService', 'Spa', 'VRDeck']
DATA['Lux'] = DATA[extras].sum(axis=1)
luxuries.append('Lux')
basic = ['FoodCourt', 'ShoppingMall']
DATA['Basic'] = DATA[basic].sum(axis=1)
luxuries.append('Basic')
DATA['LB_ratio']= DATA['Lux'] / (DATA['Basic'] + 0.1) # +0.1 for numerical stability
luxuries.append('LB_ratio')

In [ ]:
LUX = DATA[DATA.Set == 'Train'][[*luxuries,'Transported']]
LUX.head()

In [ ]:
F = pd.melt(LUX, value_vars = [*luxuries],id_vars= 'Transported')
ax = sns.displot(F, x='value', hue='Transported', col='variable', kind='kde',col_wrap= 4);
ax.set(xlim = (0,500));

# Choosing Features

In [ ]:
numeric = [*luxuries,'Age','Basic']
categorical = ['HomePlanet',"Side", "Destination","Deck"]
drop = ['CabinNumber','Cabin','Group','Position','Name', 'FirstName', 'LastName', 'GroupSize', 'FamilySize','AgeCat','GroupSizeCat','FamilySizeCat','VIP', 'ClusterSizeCat','LB_ratio','Solo']
binary = ['VIP', 'CryoSleep']
not_tested = []
exclude = ['FoodCourt', 'ShoppingMall',*extras, *not_tested]

In [ ]:
mm = StandardScaler()
DATA[numeric] = mm.fit_transform(DATA[numeric])

---

In [ ]:
fig, ax = plt.subplots(figsize=(15,15)) 
ax = sns.heatmap(DATA[DATA.Set == 'Train'][[*numeric, *binary,'Transported']].corr(),annot=True, fmt = ".2f", cmap = "coolwarm");
ax.set_title("Transported correlation to numeric features");

---

### Save features for review

In [ ]:
_HomePlanet = DATA[DATA.Set == 'Train'].HomePlanet
_Deck = DATA[DATA.Set == 'Train'].Deck
_Side = DATA[DATA.Set == 'Train'].Side
_Destination = DATA[DATA.Set == 'Train'].Destination

### Dropping unwanted columns

In [ ]:
DATA.drop([*drop, *exclude], inplace = True, axis = 1)

### Categorical to dummies

In [ ]:
#DATA = pd.get_dummies(DATA, columns=categorical, drop_first=True)
DATA = pd.get_dummies(DATA, columns=categorical)

# Split the data

In [ ]:
DATA.head()

In [ ]:
final_drop = ['PassengerId', 'Set', 'Transported', 'index']
TRAIN = DATA[DATA.Set == 'Train']
TEST = DATA[DATA.Set == 'Test']
PassengerIds = TEST.PassengerId.to_list()
TEST = TEST.drop(final_drop, axis = 1)
y = TRAIN.Transported
X = TRAIN.drop(final_drop, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = RandomSeed, stratify=y)

# Models

---

## Random Forest

In [ ]:
rf_params = {
    'n_jobs':-1,
    'random_state': RandomSeed,
    'n_estimators': 100,
    'max_depth': 4,
    'min_samples_split': 2,
    'min_samples_leaf': 2,
    'max_features': 'auto',
    'max_samples': 0.9
}

In [ ]:
# rf_grid = {
#     'max_depth': [None, 4, 6, 8],
#     'min_samples_split': [2,4],
#     'min_samples_leaf':[1,2],
#     'max_features': ['auto', 0.9],
#     'max_samples': [None, 0.9],
#     'n_estimators': [100],
# }

In [ ]:
# %%time
# rf_clf = RandomForestClassifier(**rf_params)
# rf_grid_clf = GridSearchCV(rf_clf, rf_grid, cv=3, scoring= "f1_micro")
# #rf_grid_clf = GridSearchCV(rf_clf, rf_grid, cv=3, scoring= "accuracy")
# rf_grid_clf.fit(X_train, y_train)
# print(rf_grid_clf.best_estimator_)
# print(rf_grid_clf.best_params_)
# print(rf_grid_clf.best_score_)

In [ ]:
# print("Reference score:",0.8107071636754104, 
#       " {'max_depth': 8, 'max_features': 'auto', 'max_samples': 0.9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}","\n")

# rf_scores = pd.DataFrame(rf_grid_clf.cv_results_['params'])
# rf_scores['results'] = rf_grid_clf.cv_results_['mean_test_score']
# rf_scores['std'] = rf_grid_clf.cv_results_['std_test_score']
# rf_scores = rf_scores.sort_values('results', ascending=False)
# rf_scores

In [ ]:
%%time
rf_model = RandomForestClassifier(**rf_params)
rf_model.fit(X_train, y_train)
rf_train_score = rf_model.score(X_train, y_train)
rf_accuracy = rf_model.score(X_test, y_test)
print("Train: {:.2f} %".format(rf_train_score * 100))
print("Test: {:.2f} %".format(rf_accuracy*100))
print('Overfit: {:.2f} %'.format((rf_train_score-rf_accuracy)*100))

In [ ]:
%%time
train_pred = rf_model.predict(X_train)
val_pred = rf_model.predict(X_test)
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('Random Forest Classifier', fontsize=16);

In [ ]:
features = {}
for feature, importance in zip(X_train.columns, rf_model.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame({"RF":features})
importances.sort_values("RF", ascending = False, inplace=True)
RF_best_features = list(importances[importances.RF > 0.03].index)
importances.plot.bar()
print("RF_best_features:",RF_best_features, len(RF_best_features))

---

## DNN


In [ ]:
dnn_model = Sequential()
n_cols = X.shape[1]
dnn_model.add(Input(shape = (n_cols,), name = 'input'))
# dnn_model.add(Dense(16, activation="relu"))
# dnn_model.add(Dropout(0.25))
dnn_model.add(Dense(8, activation="relu"))
dnn_model.add(Dropout(0.25))
dnn_model.add(Dense(2, activation="relu"))
dnn_model.add(BatchNormalization())
dnn_model.add(Dense(1, activation="sigmoid", name='out'))            
dnn_model.summary()

In [ ]:
tf.keras.utils.plot_model(dnn_model, show_shapes=True)

In [ ]:
%%time
dnn_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-03), metrics=['binary_accuracy'])
early_stopping_monitor = EarlyStopping(patience=25, monitor='val_binary_accuracy')
checkpoint = ModelCheckpoint("weights.hdf5", monitor = 'val_binary_accuracy', save_best_only = True)
dnn_model.fit(X_train,y_train, validation_data=(X_test,y_test), callbacks=[checkpoint, early_stopping_monitor], epochs=300, batch_size=64, verbose=0, validation_split=0.25)
dnn_model.load_weights("weights.hdf5")

In [ ]:
mtrcs = ['loss','binary_accuracy']
fig, axs = plt.subplots(1, len(mtrcs), figsize=(30,10))
for i,ax in enumerate(axs.flatten()):
    train = mtrcs[i]
    test = "val_"+mtrcs[i];
    ax.plot(dnn_model.history.history[train], label='train')
    ax.plot(dnn_model.history.history[test], label = 'test')
    ax.set_title(train)
    ax.set_xlabel('Epochs')
    ax.set_ylabel(train)
    ax.legend();

In [ ]:
_, train_dnn_accuracy = dnn_model.evaluate(X_train, y_train)
_, dnn_accuracy = dnn_model.evaluate(X_test, y_test)
print('Train accuracy: {:.2f} %'.format(train_dnn_accuracy*100))
print('Accuracy: {:.2f} %'.format(dnn_accuracy*100))
print('Overfit: {:.2f} % '.format((train_dnn_accuracy - dnn_accuracy)*100))

In [ ]:
%%time
train_pred = np.rint(rf_model.predict(X_train))
val_pred = np.rint(rf_model.predict(X_test))
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('DNN Classifier', fontsize=16);

In [ ]:
class DNN_wrapper:
    def __init__(self, model):
        self.model = model
    def predict(self, df):
        pred = np.rint(self.model.predict(df))[:,0]
        return pred.astype(np.int32)
    def predict_proba(self, df):
        probs = self.model.predict(df)
        probs2 = np.ones_like(probs) - probs
        packed = np.concatenate((probs2, probs), axis=1)        
        return packed
    
DNN_MODEL = DNN_wrapper(dnn_model)

---

## XGBoost

In [ ]:
xgb_params = {
    'random_state': RandomSeed, 
    'objective':'binary:logistic', 
    'eval_metric':['logloss'], 
    'max_depth': 4,
    'gamma': 0.25,
    'eta': 0.4,
    'reg_alpha': 0.001,
    'reg_lambda': 0.1,
    'subsample': 0.85,
    'n_estimators': 100,
    'learning_rate':0.1,
    'use_label_encoder':False
}

In [ ]:
# xgb_grid = {
#     'max_depth': [4],
#     'gamma': [0.25, 0.1, 0.01],
#     'eta': [0.4, 0.1, 1.0],
#     'reg_alpha': [0.001],
#     'reg_lambda': [0.1],
#     'subsample': [0.85, 0.9],
#     'n_estimators': [100],
#     'learning_rate':[0.1]
# }

In [ ]:
# %%time
# warnings.filterwarnings("ignore")
# xgb_clf = xgb.XGBClassifier(**xgb_params)
# xgb_grid_clf = GridSearchCV(xgb_clf, xgb_grid, cv=3, scoring= "accuracy")
# xgb_grid_clf.fit(X_train, y_train)
# print(xgb_grid_clf.best_estimator_)
# print(xgb_grid_clf.best_params_)
# print(xgb_grid_clf.best_score_)

In [ ]:
# print("Reference score:",0.81239453903973, 
#       " {'eta': 0.4, 'gamma': 0.25, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'reg_alpha': 0.001, 'reg_lambda': 0.1, 'subsample': 0.85}","\n")

# xgb_scores = pd.DataFrame(xgb_grid_clf.cv_results_['params'])
# xgb_scores['results'] = xgb_grid_clf.cv_results_['mean_test_score']
# xgb_scores['std'] = xgb_grid_clf.cv_results_['std_test_score']
# xgb_scores = xgb_scores.sort_values('results', ascending=False)
# xgb_scores

In [ ]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(X_train, y_train)
xgb_train_score = xgb_model.score(X_train, y_train)
xgb_accuracy = xgb_model.score(X_test, y_test)
print("Train: {:.2f} %".format(xgb_train_score * 100))
print("Test: {:.2f} %".format(xgb_accuracy*100))
print('Overfit: {:.2f} %'.format((xgb_train_score-xgb_accuracy)*100))

In [ ]:
%%time
train_pred = xgb_model.predict(X_train)
val_pred = xgb_model.predict(X_test)
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('XGB Classifier', fontsize=16);

---

## ExtraTrees

In [ ]:
ET_parameters = {
    'random_state': RandomSeed,
    'bootstrap': True,
    'n_estimators': 300,
    'max_depth': 8,
    'min_samples_split': 4,
    'min_samples_leaf': 1,
    'max_features': 0.9,
}

In [ ]:
# ex_param_grid = {"max_depth": [7, 8, 10], 
#                  'max_features':[0.95, 0.9, 0.85], 
#                  'n_estimators': [300],
#                  'min_samples_split': [2,4],
#                  'min_samples_leaf': [1,2],
#                  'bootstrap': [True]
#                 }

In [ ]:
# %%time
# ETC_clf = ExtraTreesClassifier(**ET_parameters)
# etc_gs = GridSearchCV(estimator = ETC_clf, param_grid = ex_param_grid, scoring='f1_micro', n_jobs=-1,verbose = 0, cv=3)
# etc_gs.fit(X_train, y_train)

# print("Best score:", etc_gs.best_score_)
# print("Best params:", etc_gs.best_params_)
# print("Best estimator:", etc_gs.best_estimator_)

In [ ]:
# print("best score:", 0.8099401748734468,
#      "{'bootstrap': True, 'max_depth': 8, 'max_features': 0.9, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 300} \n")
# etc_scores = pd.DataFrame(etc_gs.cv_results_['params'])
# etc_scores['results'] = etc_gs.cv_results_['mean_test_score']
# etc_scores['std'] = etc_gs.cv_results_['std_test_score']
# etc_scores = etc_scores.sort_values('results', ascending=False)
# etc_scores

In [ ]:
%%time
ETC_model = ExtraTreesClassifier(**ET_parameters)
ETC_model.fit(X_train, y_train)
ETC_train_score = ETC_model.score(X_train, y_train)
ETC_accuracy = ETC_model.score(X_test, y_test)
print("Train: {:.2f} %".format(ETC_train_score*100))
print("Test: {:.2f} %".format(ETC_accuracy*100))
print('Overfit: {:.2f} %'.format((ETC_train_score-ETC_accuracy)*100))

In [ ]:
%%time
train_pred = ETC_model.predict(X_train)
val_pred = ETC_model.predict(X_test)
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('ExtraTrees Classifier', fontsize=16);

In [ ]:
features = {}
for feature, importance in zip(X_train.columns, ETC_model.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame({"ETC":features})
importances.sort_values("ETC", ascending = False, inplace=True)
ETC_best_features = list(importances[importances.ETC > 0.03].index)
importances.plot.bar()
print("RF_best_features:",ETC_best_features, len(ETC_best_features))

---

## LGBM


In [ ]:
lgbm_parameters = {
    'random_state': RandomSeed,
    'max_depth':4,
    'learning_rate': 0.1,
    'subsample': 0.95,
    'reg_alpha': 0.001,
    'reg_lambda': 10,
    'n_estimators': 300,
}

In [ ]:
# lgbm_grid = {
#     'max_depth':[None, 4, 8],
#     'learning_rate': [0.01, 0.1, 0.5],
#     'subsample': [0.99, 0.95, 0.9, 0.8],
#     'reg_alpha':[0.001, 0.01, 0.1],
#     'reg_lambda': [0.01, 0.1,1.0, 10.0, 100.0],
#     'n_estimators': [100, 300, 500]
# }

In [ ]:
# %%time
# LGBM_clf = lgb.LGBMClassifier(**lgbm_parameters)
# lgbm_gs = GridSearchCV(estimator = LGBM_clf, param_grid = lgbm_grid, scoring='accuracy', n_jobs=-1,verbose = 0, cv=3)
# lgbm_gs.fit(X_train, y_train)

# print("Best score:", lgbm_gs.best_score_)
# print("Best params:", lgbm_gs.best_params_)
# print("Best estimator:", lgbm_gs.best_estimator_)

In [ ]:
# print("Reference score:",0.8116275502377666, 
#       " {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'reg_alpha': 0.001, 'reg_lambda': 10.0, 'subsample': 0.95}","\n")

# lgbm_scores = pd.DataFrame(lgbm_gs.cv_results_['params'])
# lgbm_scores['results'] = lgbm_gs.cv_results_['mean_test_score']
# lgbm_scores['std'] = lgbm_gs.cv_results_['std_test_score']
# lgbm_scores = lgbm_scores.sort_values('results', ascending=False)
# lgbm_scores

In [ ]:
%%time
LGBM_model = lgb.LGBMClassifier(**lgbm_parameters)
LGBM_model.fit(X_train, y_train)
LGBM_train_score = LGBM_model.score(X_train, y_train)
LGBM_accuracy = LGBM_model.score(X_test, y_test)
print("Train: {:.2f} %".format(LGBM_train_score*100))
print("Test: {:.2f} %".format(LGBM_accuracy*100))
print('Overfit: {:.2f} %'.format((LGBM_train_score-LGBM_accuracy)*100))

In [ ]:
%%time
train_pred = LGBM_model.predict(X_train)
val_pred = LGBM_model.predict(X_test)
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('LGBM Classifier', fontsize=16);

In [ ]:
features = {}
for feature, importance in zip(X_train.columns, LGBM_model.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame({"LGBM":features})
importances.sort_values("LGBM", ascending = False, inplace=True)
LGBM_best_features = list(importances[importances.LGBM > 0.03].index)
importances.plot.bar()
print("LGBM_best_features:",LGBM_best_features, len(LGBM_best_features))

---


## ADA
worsens LB score by a little ...

In [ ]:
DT_params = {
    'max_depth': 4,
    'max_features': 0.9,
    'min_samples_leaf': 3,
    'min_samples_split': 2
}
ADA_params = {
    'random_state': RandomSeed,
    'n_estimators': 300,
    'learning_rate': 0.01
}

In [ ]:
# ADA_grid = {
#     'base_estimator__max_depth': [None, 4, 6],
#     'base_estimator__max_features': ['auto', 0.9, 0.8],
#     'base_estimator__min_samples_split': [1,2],
#     'base_estimator__min_samples_leaf': [2,3],
#     'n_estimators': [50, 100, 300],
#     'learning_rate': [0.1, 0.01, 0.5]
# }

In [ ]:
# %%time
# ADA_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(**DT_params), **ADA_params)
# ADA_gs = GridSearchCV(estimator = ADA_clf, param_grid = ADA_grid, scoring='accuracy', n_jobs=-1,verbose = 0, cv=3)
# ADA_gs.fit(X_train, y_train)

# print("Best score:", ADA_gs.best_score_)
# print("Best params:", ADA_gs.best_params_)
# print("Best estimator:", ADA_gs.best_estimator_)

In [ ]:
# print("Reference score:",0.8084061972695199, 
#       " {'base_estimator__max_depth': 4, 'base_estimator__max_features': 'auto', 'base_estimator__min_samples_leaf': 2, 'base_estimator__min_samples_split': 2, 'learning_rate': 0.1, 'n_estimators': 100}","\n")

# ada_scores = pd.DataFrame(ADA_gs.cv_results_['params'])
# ada_scores['results'] = ADA_gs.cv_results_['mean_test_score']
# ada_scores['std'] = ADA_gs.cv_results_['std_test_score']
# ada_scores = ada_scores.sort_values('results', ascending=False)
# ada_scores

In [ ]:
%%time
ADA_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(**DT_params), **ADA_params)
ADA_model.fit(X_train,y_train)
ADA_train_score = ADA_model.score(X_train, y_train)
ADA_accuracy = ADA_model.score(X_test, y_test)
print("Train: {:.2f} %".format(ADA_train_score*100))
print("Test: {:.2f} %".format(ADA_accuracy*100))
print('Overfit: {:.2f} %'.format((ADA_train_score - ADA_accuracy)*100))

In [ ]:
%%time
train_pred = ADA_model.predict(X_train)
val_pred = ADA_model.predict(X_test)
print(classification_report(y_test, val_pred))

In [ ]:
cm = confusion_matrix(y_test, val_pred, normalize = 'pred')
cm_train = confusion_matrix(y_train, train_pred, normalize = 'pred')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels= ['Not Transported', 'Transported']);
disp_train.plot(ax=ax1, values_format='.1%', xticks_rotation='horizontal');
disp_train.ax_.set_title('Train set', {'fontsize':20});

disp_test = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels= ['Not Transported', 'Transported']);
disp_test.plot(ax=ax2, values_format='.1%', xticks_rotation='horizontal');
disp_test.ax_.set_title('Validation set',{'fontsize':20});
fig.suptitle('ADA Classifier', fontsize=16);

In [ ]:
features = {}
for feature, importance in zip(X_train.columns, ADA_model.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame({"ADA":features})
importances.sort_values("ADA", ascending = False, inplace=True)
importances
ADA_best_features = list(importances[importances.ADA > 0.03].index)
importances.plot.bar()
print("ADA_best_features:",ADA_best_features, len(ADA_best_features))

---

## SVM
worsens LB score

In [ ]:
SVM_params = {
    'random_state': RandomSeed,
    'kernel': 'rbf',
    'C': 100,
    'gamma':0.01,
    'probability': True
}

In [ ]:
# SVM_grid = {
#     'C': [1,10,100],
#     'gamma': [0.001, 0.01, 0.1]
# }

In [ ]:
# %%time
# SVM_clf = SVC(**SVM_params)
# SVM_gs = GridSearchCV(estimator = SVM_clf, param_grid = SVM_grid, scoring='accuracy', n_jobs=-1,verbose = 0, cv=3)
# SVM_gs.fit(X_train, y_train)

# print("Best score:", SVM_gs.best_score_)
# print("Best params:", SVM_gs.best_params_)
# print("Best estimator:", SVM_gs.best_estimator_)

In [ ]:
# print("Reference score:",0.8077926062279491, 
#       " {'C': 10, 'gamma': 0.1}","\n")

# svm_scores = pd.DataFrame(SVM_gs.cv_results_['params'])
# svm_scores['results'] = SVM_gs.cv_results_['mean_test_score']
# svm_scores['std'] = SVM_gs.cv_results_['std_test_score']
# svm_scores = svm_scores.sort_values('results', ascending=False)
# svm_scores

In [ ]:
%%time
SVM_model = SVC(**SVM_params)
SVM_model.fit(X_train, y_train)
svm_train_score = SVM_model.score(X_train, y_train)
SVM_accuracy = SVM_model.score(X_test, y_test)
print("Train: {:.2f} %".format(svm_train_score*100))
print("Test: {:.2f} %".format(SVM_accuracy*100))
print('Overfit: {:.2f} %'.format((svm_train_score - SVM_accuracy)*100))

---

## Scores

In [ ]:
print("Accuracy Scores:")
print("==========================================================")
print("DNN: {:.3f}".format(dnn_accuracy))
print("RandomForest: {:.3f}".format(rf_accuracy))
print("XGBoost classifier: {:.3f}".format(xgb_accuracy))
print("SVM classifier: {:.3f}".format(SVM_accuracy))
#print("LR classifier: {:.3f}".format(LR_accuracy))
#print("KNN classifier: {:.3f}".format(KNN_accuracy))
print("ADA Boost classifier: {:.3f}".format(ADA_accuracy))
print("Extra Tree classifier: {:.3f}".format(ETC_accuracy))
#print("Gradient Boosting classifier: {:.3f}".format(GBC_accuracy))
#print("Stochastic Gradient descent: {:.3f}".format(SGD_accuracy))
#print("Decision Tree classifier: {:.3f}".format(DT_accuracy))
print("LGBM classifier: {:.3f}".format(LGBM_accuracy))
print("==========================================================")

---

## Predictions

In [ ]:
# models = [rf_model, DNN_MODEL, xgb_model, ETC_model, LGBM_model, ADA_model, SVM_model]
# model_names = ["RF","DNN", "XGB", "ET", "LGBM", "ADA", "SVM"]

models = [rf_model, DNN_MODEL, xgb_model, ETC_model, LGBM_model]
model_names = ["RF","DNN", "XGB", "ET", "LGBM"]
print("using", len(models), "classifiers")

In [ ]:
TRESHOLD = 0.5000001
HVC_TRESHOLD = 0.49999

In [ ]:
TEST.head(10)

# Hard Voting Classifier (HVC)
equal weights

In [ ]:
%%time
ALL_PREDICTIONS = pd.DataFrame({'PassengerId': PassengerIds})
for i, m in enumerate(models):
    ALL_PREDICTIONS[model_names[i]] = m.predict(TEST)
ALL_PREDICTIONS['Vote'] = ALL_PREDICTIONS[model_names].mean(axis=1)
ALL_PREDICTIONS['Predict'] = ALL_PREDICTIONS.Vote.apply(lambda row: 1 if row > HVC_TRESHOLD else 0)
hvc_predictions = ALL_PREDICTIONS.Predict
ALL_PREDICTIONS.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))  
g= sns.heatmap(ALL_PREDICTIONS[model_names].corr(),annot=True, cmap = "coolwarm")

---

# Soft Voting Classifier (SVC)

In [ ]:
%%time
SVC_ALL_PREDICTIONS = pd.DataFrame({'PassengerId': PassengerIds})
for i, m in enumerate(models):
    SVC_ALL_PREDICTIONS[model_names[i]] = m.predict_proba(TEST)[:,1]
SVC_ALL_PREDICTIONS['MedianVote'] = SVC_ALL_PREDICTIONS[model_names].median(axis=1)
SVC_ALL_PREDICTIONS['SoftVote'] = SVC_ALL_PREDICTIONS[model_names].mean(axis=1)
SVC_ALL_PREDICTIONS['Predict'] = SVC_ALL_PREDICTIONS.SoftVote.apply(lambda row: 1 if row > TRESHOLD else 0)
svc_predictions = SVC_ALL_PREDICTIONS.Predict
SVC_ALL_PREDICTIONS.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))  
g= sns.heatmap(SVC_ALL_PREDICTIONS[model_names].corr(),annot=True, cmap = "coolwarm")

---

## Prediction comparison

In [ ]:
COMP_PREDICTIONS = pd.DataFrame({'PassengerId': PassengerIds})
COMP_PREDICTIONS['HVC'] = hvc_predictions
COMP_PREDICTIONS['SVC'] = svc_predictions
COMP_PREDICTIONS.head(20)

---

## Checking Train Scores - HVC

In [ ]:
TRAIN_PREDICTIONS = pd.DataFrame({'Transported':train_data.Transported,  "HomePlanet":_HomePlanet, "Lux": TRAIN.Lux, "Basic": TRAIN.Basic,
                                  'Cryo': TRAIN.CryoSleep, 
                                  "Deck": _Deck, "Side": _Side, "Dest": _Destination})
for i, m in enumerate(models):
    TRAIN_PREDICTIONS[model_names[i]] = m.predict(X)
TRAIN_PREDICTIONS['Vote'] = TRAIN_PREDICTIONS[model_names].mean(axis=1)
TRAIN_PREDICTIONS['HVC'] = TRAIN_PREDICTIONS.Vote.apply(lambda row: 1 if row > HVC_TRESHOLD else 0)
wrong = TRAIN_PREDICTIONS[TRAIN_PREDICTIONS.Transported != TRAIN_PREDICTIONS.HVC]
wrong[(wrong.Vote >= 0.4) & (wrong.Vote <= 0.6)].head(10)

In [ ]:
#just checking how many wrong predistions on TRAIN
print(1621, wrong.shape[0])

---

## Checking Train Scores - SVC

In [ ]:
SVC_TRAIN_PREDICTIONS = pd.DataFrame({'Transported':train_data.Transported, "HomePlanet":_HomePlanet, "Lux": TRAIN.Lux, "Basic": TRAIN.Basic,
                                      'Cryo': TRAIN.CryoSleep, 
                                      "Deck": _Deck, "Side": _Side, "Dest": _Destination})
for i, m in enumerate(models):
    SVC_TRAIN_PREDICTIONS[model_names[i]] = m.predict_proba(X)[:,1]
    
SVC_TRAIN_PREDICTIONS['MedianVote'] = SVC_TRAIN_PREDICTIONS[model_names].median(axis=1)
SVC_TRAIN_PREDICTIONS['SoftVote'] = SVC_TRAIN_PREDICTIONS[model_names].mean(axis=1)
SVC_TRAIN_PREDICTIONS['SVC'] = SVC_TRAIN_PREDICTIONS.SoftVote.apply(lambda row: 1 if row > TRESHOLD else 0)
TRAIN_PREDICTIONS['SVC'] = SVC_TRAIN_PREDICTIONS['SVC']

wrong = SVC_TRAIN_PREDICTIONS[SVC_TRAIN_PREDICTIONS.Transported != SVC_TRAIN_PREDICTIONS.SVC]
#wrong.head(20)
WT = wrong[(wrong.SoftVote >= 0.35) & (wrong.SoftVote <= 0.65) & (wrong.Transported == True)].sort_values("SoftVote", ascending=False)
WS = wrong[(wrong.SoftVote >= 0.35) & (wrong.SoftVote <= 0.65) & (wrong.Transported == False)].sort_values("SoftVote", ascending=True)
#counting wrong predictions on TRAIN
print(1637, wrong.shape[0])

In [ ]:
print(257, WT.shape[0])
WT.head(10)

In [ ]:
print(570, WS.shape[0])
WS.head(10)

In [ ]:
print(827, WS.shape[0] +WT.shape[0])

---

In [ ]:
train_scores = dict()
for clf in [*model_names, 'HVC','SVC']:
    train_scores[clf] = [len(TRAIN_PREDICTIONS[TRAIN_PREDICTIONS.Transported == TRAIN_PREDICTIONS[clf]]) / TRAIN_PREDICTIONS.shape[0]]

TRAIN_SCORES = pd.DataFrame(train_scores)
TRAIN_SCORES

In [ ]:
TRAIN_SCORES.plot.bar();

---

# Submission

In [ ]:
#output = pd.DataFrame({'PassengerId': PassengerIds, 'Transported': DNN_MODEL.predict(TEST)})
#output = pd.DataFrame({'PassengerId': PassengerIds, 'Transported': rf_model.predict(TEST)})
#output = pd.DataFrame({'PassengerId': PassengerIds, 'Transported': xgb_model.predict(TEST)})
output = pd.DataFrame({'PassengerId': PassengerIds, 'Transported': svc_predictions})
#output = pd.DataFrame({'PassengerId': PassengerIds, 'Transported': hvc_predictions})
output.Transported = output.Transported.astype(bool)
output

In [ ]:
#output
output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

---

In [ ]:
end_time = time.time()
print("Notebook run time: {:.1f} seconds. Finished at {}".format(end_time - start_time, datetime.now()) )